In [1]:
import tensorflow as tf
import numpy as np

In [2]:
with np.load("notMNIST.npz") as data:
    Data, Target = data ["images"], data["labels"]
    Data = np.reshape(Data, [-1, 28*28])
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [3]:
num_label = 10
num_feature = 28*28
train_size = len(trainData)

In [4]:
with tf.Session() as s:
    testTarget_oh = s.run(tf.one_hot(testTarget, num_label))
    trainTarget_oh = s.run(tf.one_hot(trainTarget, num_label))
    validTarget_oh = s.run(tf.one_hot(validTarget, num_label))

In [5]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100 # how many samples you load at a time for training

# height x width
x = tf.placeholder('float', [None, 784]) # image is flattened 28 x 28
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {
            'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
            'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))
    }

    hidden_2_layer = {
            'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
            'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))
    }

    hidden_3_layer = {
            'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
            'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))
    }

    output_layer = {
            'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
            'biases': tf.Variable(tf.random_normal([n_classes]))
    }

    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1) # rectified linear

    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output_layer = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])

    return output_layer

In [10]:
def train_neural_network(x, y):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

    # learning_rate = 0.001 default
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    # cycles of feed forward + back prop
    hm_epochs = 30

    with tf.Session() as s:
        tf.global_variables_initializer().run()

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for step in range(int(train_size/batch_size)):
                offset = (step*batch_size) % train_size
                batch_data = trainData[offset:(offset + batch_size)]
                batch_target = trainTarget_oh[offset:(offset + batch_size)]

                _, c = s.run([optimizer, cost], feed_dict={x: batch_data, y: batch_target})
                epoch_loss += c

            print 'Epoch: ', epoch, ' completed out of ', hm_epochs, ' loss: ', epoch_loss
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print 'Accuracy: ', accuracy.eval({x: testData, y: testTarget_oh})

In [11]:
train_neural_network(x, y)

Epoch:  0  completed out of  30  loss:  1409794.4208984375
Epoch:  1  completed out of  30  loss:  521651.1258544922
Epoch:  2  completed out of  30  loss:  340851.69396972656
Epoch:  3  completed out of  30  loss:  237617.17849731445
Epoch:  4  completed out of  30  loss:  164086.09784698486
Epoch:  5  completed out of  30  loss:  117412.27304840088
Epoch:  6  completed out of  30  loss:  85733.13893890381
Epoch:  7  completed out of  30  loss:  63459.37327384949
Epoch:  8  completed out of  30  loss:  51910.42724323273
Epoch:  9  completed out of  30  loss:  37509.52601838112
Epoch:  10  completed out of  30  loss:  30148.86616563797
Epoch:  11  completed out of  30  loss:  24361.44171075523
Epoch:  12  completed out of  30  loss:  19049.981132745743
Epoch:  13  completed out of  30  loss:  14580.081715792418
Epoch:  14  completed out of  30  loss:  15730.572695851326
Epoch:  15  completed out of  30  loss:  11037.367868334055
Epoch:  16  completed out of  30  loss:  12442.6176774501